In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spamdat/realspambase.data


In [10]:
df=pd.read_csv('/kaggle/input/spamdat/realspambase.data')

In [11]:
df.columns

Index(['0', '0.64', '0.64.1', '0.1', '0.32', '0.2', '0.3', '0.4', '0.5', '0.6',
       '0.7', '0.64.2', '0.8', '0.9', '0.10', '0.32.1', '0.11', '1.29', '1.93',
       '0.12', '0.96', '0.13', '0.14', '0.15', '0.16', '0.17', '0.18', '0.19',
       '0.20', '0.21', '0.22', '0.23', '0.24', '0.25', '0.26', '0.27', '0.28',
       '0.29', '0.30', '0.31', '0.32.2', '0.33', '0.34', '0.35', '0.36',
       '0.37', '0.38', '0.39', '0.40', '0.41', '0.42', '0.778', '0.43', '0.44',
       '3.756', '61', '278', '1'],
      dtype='object')

In [12]:
df["1"].value_counts()

0    2788
1    1812
Name: 1, dtype: int64

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 58 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       4600 non-null   float64
 1   0.64    4600 non-null   float64
 2   0.64.1  4600 non-null   float64
 3   0.1     4600 non-null   float64
 4   0.32    4600 non-null   float64
 5   0.2     4600 non-null   float64
 6   0.3     4600 non-null   float64
 7   0.4     4600 non-null   float64
 8   0.5     4600 non-null   float64
 9   0.6     4600 non-null   float64
 10  0.7     4600 non-null   float64
 11  0.64.2  4600 non-null   float64
 12  0.8     4600 non-null   float64
 13  0.9     4600 non-null   float64
 14  0.10    4600 non-null   float64
 15  0.32.1  4600 non-null   float64
 16  0.11    4600 non-null   float64
 17  1.29    4600 non-null   float64
 18  1.93    4600 non-null   float64
 19  0.12    4600 non-null   float64
 20  0.96    4600 non-null   float64
 21  0.13    4600 non-null   float64
 22  

In [14]:
dict_1={}

dict_1=dict(df.corr()['1'])

In [15]:
list_features=[]
for key,values in dict_1.items():
    if abs(values)<0.2:
        list_features.append(key)

In [16]:
#lets drop the features which have coorelation less than <0.2
df=df.drop(list_features,axis=1)

In [17]:
y=df['1']
x=df.drop(['1'],axis=1)

In [18]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=0,test_size=0.2)

In [20]:
from imblearn.over_sampling import SMOTE
print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(x_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))

Before OverSampling, counts of label '1': 1430
Before OverSampling, counts of label '0': 2250 

After OverSampling, the shape of train_X: (4500, 19)
After OverSampling, the shape of train_y: (4500,) 

After OverSampling, counts of label '1': 2250
After OverSampling, counts of label '0': 2250


In [21]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)
from sklearn.metrics import f1_score

2.4.1


In [26]:
from statistics import stdev

In [44]:
def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

In [45]:
Xnb=X_train_res.to_numpy()

In [46]:
Xnk,ynk=shuffle_in_unison(Xnb, y_train_res)

NEURAL NETWORK FUNCTION:

In [185]:
def callf2(xx,yy,xt,yt):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    
    model.compile(optimizer='adam',
                  loss='mean_absolute_error',
                  metrics=['accuracy'])
    model.fit(xx, yy , epochs=90)
    ls=[]
    test_loss, test_acc = model.evaluate(xt,  yt, verbose=2)
    print('\nTest accuracy:', test_acc)
    tr_loss, tr_acc = model.evaluate(xx,  yy, verbose=2)
    ls.append(test_acc)
    ls.append(tr_acc)
    ypr=model.predict(xt)
    ypr=(ypr>0.5)*1
    ypre= np.ravel(ypr)
    ls.append(f1_score(yt, ypre))
    return ls

NEURAL NETWORK ON NON SMOTE DATA:

In [186]:
ste=[]
sta=[]
sf=[]
for i in range(30):
    r=callf2(x_train,y_train,x_test,y_test)
    ste.append(r[0])
    sta.append(r[1])
    sf.append(r[2])

Epoch 1/90
115/115 [==============================] - 1s 2ms/step - loss: 7.0759 - accuracy: 0.5710
Epoch 2/90
115/115 [==============================] - 0s 2ms/step - loss: 1.8352 - accuracy: 0.6522
Epoch 3/90
115/115 [==============================] - 0s 2ms/step - loss: 1.3219 - accuracy: 0.7147
Epoch 4/90
115/115 [==============================] - 0s 2ms/step - loss: 1.0010 - accuracy: 0.7302
Epoch 5/90
115/115 [==============================] - 0s 2ms/step - loss: 1.2601 - accuracy: 0.7187
Epoch 6/90
115/115 [==============================] - 0s 2ms/step - loss: 1.3258 - accuracy: 0.7217
Epoch 7/90
115/115 [==============================] - 0s 2ms/step - loss: 1.5361 - accuracy: 0.7104
Epoch 8/90
115/115 [==============================] - 0s 2ms/step - loss: 2.4950 - accuracy: 0.6491
Epoch 9/90
115/115 [==============================] - 0s 2ms/step - loss: 1.0520 - accuracy: 0.7366
Epoch 10/90
115/115 [==============================] - 0s 2ms/step - loss: 0.9586 - accuracy: 0.7560

115/115 [==============================] - 0s 2ms/step - loss: 0.1870 - accuracy: 0.8859
Epoch 83/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1669 - accuracy: 0.9073
Epoch 84/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1886 - accuracy: 0.8790
Epoch 85/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1869 - accuracy: 0.8761
Epoch 86/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1745 - accuracy: 0.8957
Epoch 87/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1773 - accuracy: 0.8924
Epoch 88/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1643 - accuracy: 0.9053
Epoch 89/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1883 - accuracy: 0.8814
Epoch 90/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1737 - accuracy: 0.8858
29/29 - 0s - loss: 0.1805 - accuracy: 0.8902

Test accuracy: 0.8902173638343811
115/115 - 0s - loss: 0.

115/115 [==============================] - 0s 2ms/step - loss: 0.2064 - accuracy: 0.8724
Epoch 73/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2389 - accuracy: 0.8810
Epoch 74/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1957 - accuracy: 0.8933
Epoch 75/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2180 - accuracy: 0.8754
Epoch 76/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2296 - accuracy: 0.8764
Epoch 77/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2021 - accuracy: 0.8889
Epoch 78/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1920 - accuracy: 0.8933
Epoch 79/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1927 - accuracy: 0.8911
Epoch 80/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2151 - accuracy: 0.8690
Epoch 81/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2003 - accuracy: 0.8872
Ep

115/115 [==============================] - 0s 2ms/step - loss: 0.2129 - accuracy: 0.8716
Epoch 63/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2349 - accuracy: 0.8658
Epoch 64/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2206 - accuracy: 0.8642
Epoch 65/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2198 - accuracy: 0.8814
Epoch 66/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2017 - accuracy: 0.8845
Epoch 67/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1980 - accuracy: 0.8817
Epoch 68/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2204 - accuracy: 0.8683
Epoch 69/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2323 - accuracy: 0.8664
Epoch 70/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1836 - accuracy: 0.8955
Epoch 71/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1845 - accuracy: 0.8953
Ep

115/115 [==============================] - 0s 2ms/step - loss: 0.2328 - accuracy: 0.8686
Epoch 53/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2423 - accuracy: 0.8807
Epoch 54/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2562 - accuracy: 0.8600
Epoch 55/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2741 - accuracy: 0.8597
Epoch 56/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2492 - accuracy: 0.8706
Epoch 57/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2723 - accuracy: 0.8547
Epoch 58/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2522 - accuracy: 0.8569
Epoch 59/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2140 - accuracy: 0.8719
Epoch 60/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2336 - accuracy: 0.8659
Epoch 61/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2233 - accuracy: 0.8729
Ep

...

115/115 [==============================] - 0s 2ms/step - loss: 0.2129 - accuracy: 0.8808
Epoch 74/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1919 - accuracy: 0.8904
Epoch 75/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1949 - accuracy: 0.8857
Epoch 76/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2003 - accuracy: 0.8833
Epoch 77/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2164 - accuracy: 0.8762
Epoch 78/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1947 - accuracy: 0.8869
Epoch 79/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1972 - accuracy: 0.8914
Epoch 80/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1958 - accuracy: 0.8852
Epoch 81/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2219 - accuracy: 0.8657
Epoch 82/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1990 - accuracy: 0.8819
Ep

115/115 [==============================] - 0s 2ms/step - loss: 0.2045 - accuracy: 0.8805
Epoch 64/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1923 - accuracy: 0.8924
Epoch 65/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2025 - accuracy: 0.8853
Epoch 66/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2077 - accuracy: 0.8939
Epoch 67/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2361 - accuracy: 0.8725
Epoch 68/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1838 - accuracy: 0.8958
Epoch 69/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2156 - accuracy: 0.8767
Epoch 70/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1830 - accuracy: 0.8954
Epoch 71/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1826 - accuracy: 0.9003
Epoch 72/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2212 - accuracy: 0.8733
Ep

115/115 [==============================] - 0s 2ms/step - loss: 0.1982 - accuracy: 0.8826
Epoch 54/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2044 - accuracy: 0.8792
Epoch 55/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2239 - accuracy: 0.8750
Epoch 56/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2096 - accuracy: 0.8836
Epoch 57/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2318 - accuracy: 0.8612
Epoch 58/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2105 - accuracy: 0.8859
Epoch 59/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2278 - accuracy: 0.8738
Epoch 60/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1988 - accuracy: 0.8885
Epoch 61/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2040 - accuracy: 0.8752
Epoch 62/90
115/115 [==============================] - 0s 2ms/step - loss: 0.1894 - accuracy: 0.8926
Ep

115/115 [==============================] - 0s 2ms/step - loss: 0.2332 - accuracy: 0.8610
Epoch 44/90
115/115 [==============================] - 0s 2ms/step - loss: 0.3306 - accuracy: 0.8467
Epoch 45/90
115/115 [==============================] - 0s 2ms/step - loss: 0.4089 - accuracy: 0.8182
Epoch 46/90
115/115 [==============================] - 0s 2ms/step - loss: 0.3093 - accuracy: 0.8407
Epoch 47/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2458 - accuracy: 0.8734
Epoch 48/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2763 - accuracy: 0.8405
Epoch 49/90
115/115 [==============================] - 0s 2ms/step - loss: 0.3574 - accuracy: 0.8525
Epoch 50/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2834 - accuracy: 0.8522
Epoch 51/90
115/115 [==============================] - 0s 2ms/step - loss: 0.3343 - accuracy: 0.8400
Epoch 52/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2625 - accuracy: 0.8535
Ep

115/115 [==============================] - 0s 2ms/step - loss: 0.4136 - accuracy: 0.8324
Epoch 34/90
115/115 [==============================] - 0s 2ms/step - loss: 0.3663 - accuracy: 0.8198
Epoch 35/90
115/115 [==============================] - 0s 2ms/step - loss: 0.3851 - accuracy: 0.8332
Epoch 36/90
115/115 [==============================] - 0s 2ms/step - loss: 0.4887 - accuracy: 0.7975
Epoch 37/90
115/115 [==============================] - 0s 2ms/step - loss: 0.4335 - accuracy: 0.8275
Epoch 38/90
115/115 [==============================] - 0s 2ms/step - loss: 0.3265 - accuracy: 0.8462
Epoch 39/90
115/115 [==============================] - 0s 2ms/step - loss: 0.4099 - accuracy: 0.8109
Epoch 40/90
115/115 [==============================] - 0s 2ms/step - loss: 0.4723 - accuracy: 0.8152
Epoch 41/90
115/115 [==============================] - 0s 2ms/step - loss: 0.4406 - accuracy: 0.8129
Epoch 42/90
115/115 [==============================] - 0s 2ms/step - loss: 0.2978 - accuracy: 0.8398
Ep

In [187]:
sum(sta)/30

0.9052264432112376

In [188]:
sum(ste)/30

0.9027898589769999

In [189]:
sum(sf)/30

0.8755498556788177

In [190]:
max(sta)

0.925000011920929

In [191]:
max(ste)

0.9315217137336731

In [192]:
max(sf)

0.9147496617050067

In [193]:
stdev(sta)

0.01400675796985451

In [194]:
stdev(ste)

0.014379598438667254

In [195]:
stdev(sf)

0.022392536618493965

NEURAL NETWORK WITH SMOTE OVERSAMPLE:

In [196]:
ste2=[]
sta2=[]
sf2=[]
for i in range(30):
    r=callf2(Xnk,ynk,x_test,y_test)
    ste2.append(r[0])
    sta2.append(r[1])
    sf2.append(r[2])

Epoch 1/90
141/141 [==============================] - 1s 2ms/step - loss: 4.5585 - accuracy: 0.5997
Epoch 2/90
141/141 [==============================] - 0s 2ms/step - loss: 3.4284 - accuracy: 0.6223
Epoch 3/90
141/141 [==============================] - 0s 2ms/step - loss: 2.1137 - accuracy: 0.6633
Epoch 4/90
141/141 [==============================] - 0s 2ms/step - loss: 2.0184 - accuracy: 0.6716
Epoch 5/90
141/141 [==============================] - 0s 2ms/step - loss: 1.1071 - accuracy: 0.7430
Epoch 6/90
141/141 [==============================] - 0s 2ms/step - loss: 0.8432 - accuracy: 0.7517
Epoch 7/90
141/141 [==============================] - 0s 2ms/step - loss: 1.2008 - accuracy: 0.6984
Epoch 8/90
141/141 [==============================] - 0s 2ms/step - loss: 0.9545 - accuracy: 0.7399
Epoch 9/90
141/141 [==============================] - 0s 2ms/step - loss: 0.8706 - accuracy: 0.7487
Epoch 10/90
141/141 [==============================] - 0s 2ms/step - loss: 0.6939 - accuracy: 0.7563

141/141 [==============================] - 0s 2ms/step - loss: 0.1315 - accuracy: 0.9188
Epoch 83/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1179 - accuracy: 0.9254
Epoch 84/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1377 - accuracy: 0.9192
Epoch 85/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1272 - accuracy: 0.9343
Epoch 86/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1319 - accuracy: 0.9204
Epoch 87/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1248 - accuracy: 0.9271
Epoch 88/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1204 - accuracy: 0.9261
Epoch 89/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1111 - accuracy: 0.9265
Epoch 90/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1285 - accuracy: 0.9191
29/29 - 0s - loss: 0.1333 - accuracy: 0.9163

Test accuracy: 0.916304349899292
141/141 - 0s - loss: 0.1

141/141 [==============================] - 0s 2ms/step - loss: 0.1314 - accuracy: 0.9201
Epoch 73/90
141/141 [==============================] - 0s 3ms/step - loss: 0.1558 - accuracy: 0.9102
Epoch 74/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1483 - accuracy: 0.9160
Epoch 75/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1386 - accuracy: 0.9176
Epoch 76/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1513 - accuracy: 0.9185
Epoch 77/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1355 - accuracy: 0.9172
Epoch 78/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1242 - accuracy: 0.9304
Epoch 79/90
141/141 [==============================] - 0s 3ms/step - loss: 0.1263 - accuracy: 0.9237
Epoch 80/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1175 - accuracy: 0.9271
Epoch 81/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1179 - accuracy: 0.9264
Ep

141/141 [==============================] - 0s 2ms/step - loss: 0.1757 - accuracy: 0.9142
Epoch 63/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1718 - accuracy: 0.9197
Epoch 64/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1843 - accuracy: 0.9049
Epoch 65/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2017 - accuracy: 0.8996
Epoch 66/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1744 - accuracy: 0.9098
Epoch 67/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1574 - accuracy: 0.9134
Epoch 68/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1619 - accuracy: 0.9155
Epoch 69/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1646 - accuracy: 0.9126
Epoch 70/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1505 - accuracy: 0.9213
Epoch 71/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1739 - accuracy: 0.9122
Ep

141/141 [==============================] - 0s 2ms/step - loss: 0.2589 - accuracy: 0.8618
Epoch 53/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2740 - accuracy: 0.8637
Epoch 54/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2165 - accuracy: 0.8869
Epoch 55/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2180 - accuracy: 0.8900
Epoch 56/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1983 - accuracy: 0.9080
Epoch 57/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2176 - accuracy: 0.8882
Epoch 58/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2213 - accuracy: 0.8918
Epoch 59/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2325 - accuracy: 0.8850
Epoch 60/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2151 - accuracy: 0.8900
Epoch 61/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2089 - accuracy: 0.8917
Ep

...

141/141 [==============================] - 1s 4ms/step - loss: 0.1828 - accuracy: 0.9062
Epoch 71/90
141/141 [==============================] - 1s 4ms/step - loss: 0.1669 - accuracy: 0.9110
Epoch 72/90
141/141 [==============================] - 1s 4ms/step - loss: 0.1655 - accuracy: 0.9157
Epoch 73/90
141/141 [==============================] - 1s 4ms/step - loss: 0.1743 - accuracy: 0.9122
Epoch 74/90
141/141 [==============================] - 1s 4ms/step - loss: 0.1693 - accuracy: 0.9033
Epoch 75/90
141/141 [==============================] - 1s 4ms/step - loss: 0.1584 - accuracy: 0.9153
Epoch 76/90
141/141 [==============================] - 0s 3ms/step - loss: 0.1610 - accuracy: 0.9134
Epoch 77/90
141/141 [==============================] - 1s 4ms/step - loss: 0.1578 - accuracy: 0.9157
Epoch 78/90
141/141 [==============================] - 1s 4ms/step - loss: 0.1522 - accuracy: 0.9148
Epoch 79/90
141/141 [==============================] - 1s 4ms/step - loss: 0.1510 - accuracy: 0.9094
Ep

141/141 [==============================] - 0s 2ms/step - loss: 0.1611 - accuracy: 0.9049
Epoch 61/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1795 - accuracy: 0.8982
Epoch 62/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1687 - accuracy: 0.9021
Epoch 63/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1656 - accuracy: 0.9062
Epoch 64/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2064 - accuracy: 0.8795
Epoch 65/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1554 - accuracy: 0.9066
Epoch 66/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1544 - accuracy: 0.9064
Epoch 67/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1548 - accuracy: 0.9120
Epoch 68/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1516 - accuracy: 0.9059
Epoch 69/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1500 - accuracy: 0.9089
Ep

141/141 [==============================] - 0s 2ms/step - loss: 0.2258 - accuracy: 0.8772
Epoch 50/90
141/141 [==============================] - 0s 3ms/step - loss: 0.2044 - accuracy: 0.8924
Epoch 51/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2152 - accuracy: 0.8855
Epoch 52/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2012 - accuracy: 0.9010
Epoch 53/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2057 - accuracy: 0.8969
Epoch 54/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2368 - accuracy: 0.8822
Epoch 55/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2091 - accuracy: 0.8914
Epoch 56/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2198 - accuracy: 0.8883
Epoch 57/90
141/141 [==============================] - 0s 3ms/step - loss: 0.2077 - accuracy: 0.8900
Epoch 58/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1806 - accuracy: 0.8980
Ep

141/141 [==============================] - 0s 2ms/step - loss: 0.2900 - accuracy: 0.8556
Epoch 40/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2349 - accuracy: 0.8888
Epoch 41/90
141/141 [==============================] - 0s 2ms/step - loss: 0.3051 - accuracy: 0.8584
Epoch 42/90
141/141 [==============================] - 0s 2ms/step - loss: 0.3764 - accuracy: 0.8185
Epoch 43/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2293 - accuracy: 0.8871
Epoch 44/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2901 - accuracy: 0.8775
Epoch 45/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2528 - accuracy: 0.8814
Epoch 46/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2721 - accuracy: 0.8675
Epoch 47/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2451 - accuracy: 0.8784
Epoch 48/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2536 - accuracy: 0.8722
Ep

141/141 [==============================] - 0s 3ms/step - loss: 0.3079 - accuracy: 0.8591
Epoch 30/90
141/141 [==============================] - 0s 3ms/step - loss: 0.3714 - accuracy: 0.8342
Epoch 31/90
141/141 [==============================] - 0s 3ms/step - loss: 0.2821 - accuracy: 0.8526
Epoch 32/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2634 - accuracy: 0.8671
Epoch 33/90
141/141 [==============================] - 0s 2ms/step - loss: 0.3359 - accuracy: 0.8429
Epoch 34/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2506 - accuracy: 0.8666
Epoch 35/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2831 - accuracy: 0.8589
Epoch 36/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2372 - accuracy: 0.8716
Epoch 37/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2886 - accuracy: 0.8588
Epoch 38/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2696 - accuracy: 0.8586
Ep

In [197]:
sum(sta2)/30

0.9277333358923594

In [198]:
sum(ste2)/30

0.9157246351242065

In [199]:
sum(sf2)/30

0.8991019502540668

In [200]:
max(sta2)

0.937333345413208

In [201]:
max(ste2)

0.9336956739425659

In [202]:
max(sf2)

0.9202614379084968

In [203]:
stdev(sta2)

0.009957667854307324

In [204]:
stdev(ste2)

0.010127335570038986

In [205]:
stdev(sf2)

0.012258542318234644

GAN MODEL:

In [71]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [72]:
def get_generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace=True),
    )
class Generator(nn.Module):

    def __init__(self, z_dim=19, im_dim=19, hidden_dim=128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            get_generator_block(z_dim, hidden_dim),
            get_generator_block(hidden_dim, hidden_dim * 2),
            get_generator_block(hidden_dim * 2, hidden_dim * 4),
            get_generator_block(hidden_dim * 4, hidden_dim * 8),
            nn.Linear(hidden_dim * 8, im_dim),
            nn.Sigmoid()
        )
    def forward(self, noise):
        return self.gen(noise)
    
    # Needed for grading
    def get_gen(self):

        return self.gen
def get_discriminator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.LeakyReLU(0.2, inplace=True)        
    )
class Discriminator(nn.Module):
    def __init__(self, im_dim=19, hidden_dim=128):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            get_discriminator_block(im_dim, hidden_dim * 4),
            get_discriminator_block(hidden_dim * 4, hidden_dim * 2),
            get_discriminator_block(hidden_dim * 2, hidden_dim),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, image):

        return self.disc(image)
    
    def get_disc(self):

        return self.dis
    

In [73]:
Xtb=x_train.to_numpy()

In [74]:
X_oversampled=Xnb[3680:]

In [75]:
X_oversampled.shape

(820, 19)

In [76]:
l=[1]*820
y_oversampled=np.array(l)
y_oversampled.shape

(820,)

In [77]:
X_oversampled = torch.from_numpy(X_oversampled)

In [78]:
criterion = nn.BCEWithLogitsLoss()
n_epochs = 400
z_dim = 19
batch_size = 128
lr = 0.00001
display_step = 500
device = 'cpu'

In [79]:
gen = Generator(z_dim).to(device)
gen_opt = torch.optim.Adam(gen.parameters(), lr=lr)
disc = Discriminator().to(device) 
disc_opt = torch.optim.Adam(disc.parameters(), lr=lr)

In [80]:
def get_disc_loss(gen, disc, criterion, real, device):

    fake = gen(X_oversampled.float())
    disc_fake_pred = disc(fake.detach())
    disc_fake_loss = criterion(disc_fake_pred, torch.zeros_like(disc_fake_pred))
    disc_real_pred = disc(real)
    disc_real_loss = criterion(disc_real_pred, torch.ones_like(disc_real_pred))
    disc_loss = (disc_fake_loss + disc_real_loss) / 2
    return disc_loss

In [81]:
def get_gen_loss(gen, disc, criterion, num_images, z_dim, device):
    fake_images = gen(X_oversampled.float())
    
    disc_fake_pred = disc(fake_images)
    gen_loss = criterion(disc_fake_pred, torch.ones_like(disc_fake_pred))
    return gen_loss

In [83]:
ytr=y_train.ravel()

In [84]:
li=[]
for i in range(len(ytr)):
    if int(ytr[i])==1:
        li.append(Xtb[i])

In [85]:
X_real=np.array(li)

In [86]:
X_real.shape

(1430, 19)

In [87]:
li2=[1]*1430

In [88]:
y_real=np.array(li2)
y_real.shape

(1430,)

In [89]:
from torch.utils.data import TensorDataset, DataLoader
tensor_x = torch.Tensor(X_real) 
tensor_y = torch.Tensor(y_real)
my_dataset = TensorDataset(tensor_x,tensor_y) # create your datset

In [90]:
dataloader = DataLoader(
    my_dataset,
    batch_size=batch_size,
    shuffle=True)

In [91]:
cur_step = 0
mean_generator_loss = 0
mean_discriminator_loss = 0
test_generator = True 
gen_loss = False
error = False
for epoch in range(n_epochs):
  
    # Dataloader returns the batches
    for real, _ in tqdm(dataloader):
        cur_batch_size = len(real)

        # Flatten the batch of real images from the dataset
        real = real.view(cur_batch_size, -1).to(device)

        ### Update discriminator ###
        # Zero out the gradients before backpropagation
        disc_opt.zero_grad()

        # Calculate discriminator loss
        disc_loss = get_disc_loss(gen, disc, criterion, real, device)

        # Update gradients#
        disc_loss.backward(retain_graph=True)

        # Update optimizer
        disc_opt.step()

        # For testing purposes, to keep track of the generator weights
        if test_generator:
            old_generator_weights = gen.gen[0][0].weight.detach().clone()

        ### Update generator ###
        #     Hint: This code will look a lot like the discriminator updates!
        #     These are the steps you will need to complete:
        #       1) Zero out the gradients.
        #       2) Calculate the generator loss, assigning it to gen_loss.
        #       3) Backprop through the generator: update the gradients and optimizer.
        #### START CODE HERE ####
        gen_opt.zero_grad()
        gen_loss = get_gen_loss(gen, disc, criterion, cur_batch_size, z_dim, device)
        gen_loss.backward()
        gen_opt.step()        
        #### END CODE HERE ####

        # For testing purposes, to check that your code changes the generator weights
        if test_generator:
            try:
                assert lr > 0.0000002 or (gen.gen[0][0].weight.grad.abs().max() < 0.0005 and epoch == 0)
                assert torch.any(gen.gen[0][0].weight.detach().clone() != old_generator_weights)
            except:
                error = True
                print("Runtime tests have failed")

        # Keep track of the average discriminator loss
        mean_discriminator_loss += disc_loss.item() / display_step

        # Keep track of the average generator loss
        mean_generator_loss += gen_loss.item() / display_step

        ### Visualization code ###
        if cur_step % display_step == 0 and cur_step > 0:
            print(f"Epoch {epoch}, step {cur_step}: Generator loss: {mean_generator_loss}, discriminator loss: {mean_discriminator_loss}")
            #fake_noise = get_noise(cur_batch_size, z_dim, device=device)
            #fake = gen(fake_noise)
            #show_tensor_images(fake)
            #show_tensor_images(real)
            mean_generator_loss = 0
            mean_discriminator_loss = 0
        cur_step += 1


  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

...

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

In [92]:
res=gen(X_oversampled.float())

In [93]:
res

tensor([[1.9769e-03, 1.2337e-04, 1.4418e-04,  ..., 5.2173e-05, 9.8546e-01,
         9.9977e-01],
        [1.7004e-03, 1.0953e-04, 1.3373e-04,  ..., 4.5598e-05, 9.8383e-01,
         9.9979e-01],
        [1.6518e-03, 8.3618e-05, 1.0606e-04,  ..., 3.5250e-05, 9.8555e-01,
         9.9984e-01],
        ...,
        [5.1444e-05, 6.0257e-06, 1.9980e-05,  ..., 3.5645e-06, 1.9055e-06,
         9.9992e-01],
        [9.9439e-01, 1.2033e-05, 2.8433e-05,  ..., 1.0416e-05, 9.8746e-01,
         9.9997e-01],
        [2.0369e-05, 2.6084e-06, 4.1035e-06,  ..., 1.0492e-06, 9.6456e-08,
         9.9999e-01]], grad_fn=<SigmoidBackward>)

In [94]:
X_oversampled

tensor([[8.9509e-01, 0.0000e+00, 1.0360e-01,  ..., 4.3152e-02, 3.0000e+00,
         5.7000e+01],
        [9.0978e-01, 0.0000e+00, 7.5155e-01,  ..., 0.0000e+00, 7.0000e+00,
         3.6000e+01],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 1.0000e+00,
         2.0000e+00],
        ...,
        [1.2000e-01, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 4.1000e+01,
         4.8700e+02],
        [2.0658e-01, 3.7074e-01, 1.9561e-01,  ..., 1.9261e-01, 1.0500e+02,
         1.0260e+03],
        [1.9638e-01, 0.0000e+00, 6.4036e-02,  ..., 2.8031e-01, 4.7000e+01,
         2.7300e+02]], dtype=torch.float64)

In [95]:
fres=res.detach().numpy()
fres.shape

(820, 19)

GAN+SMOTE DATA:

In [102]:
fin=np.concatenate((Xnb[:3680], fres), axis=0)

In [103]:
def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

In [104]:
Xnew,ynew=shuffle_in_unison(fin, y_train_res)

In [206]:
ste3=[]
sta3=[]
sf3=[]
for i in range(30):
    r=callf2(Xnew,ynew,x_test,y_test)
    ste3.append(r[0])
    sta3.append(r[1])
    sf3.append(r[2])

Epoch 1/90
141/141 [==============================] - 1s 2ms/step - loss: 4.1241 - accuracy: 0.5865
Epoch 2/90
141/141 [==============================] - 0s 2ms/step - loss: 2.4426 - accuracy: 0.7092
Epoch 3/90
141/141 [==============================] - 0s 2ms/step - loss: 2.0507 - accuracy: 0.7091
Epoch 4/90
141/141 [==============================] - 0s 2ms/step - loss: 0.8089 - accuracy: 0.7815
Epoch 5/90
141/141 [==============================] - 0s 2ms/step - loss: 1.0454 - accuracy: 0.7532
Epoch 6/90
141/141 [==============================] - 0s 2ms/step - loss: 0.9443 - accuracy: 0.7605
Epoch 7/90
141/141 [==============================] - 0s 2ms/step - loss: 0.7135 - accuracy: 0.7963
Epoch 8/90
141/141 [==============================] - 0s 2ms/step - loss: 0.5935 - accuracy: 0.8029
Epoch 9/90
141/141 [==============================] - 0s 2ms/step - loss: 0.9184 - accuracy: 0.7677
Epoch 10/90
141/141 [==============================] - 0s 2ms/step - loss: 0.6742 - accuracy: 0.8005

141/141 [==============================] - 0s 2ms/step - loss: 0.1370 - accuracy: 0.9206
Epoch 82/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1368 - accuracy: 0.9188
Epoch 83/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1421 - accuracy: 0.9113
Epoch 84/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1364 - accuracy: 0.9196
Epoch 85/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1332 - accuracy: 0.9204
Epoch 86/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1334 - accuracy: 0.9184
Epoch 87/90
141/141 [==============================] - 0s 3ms/step - loss: 0.1336 - accuracy: 0.9195
Epoch 88/90
141/141 [==============================] - 0s 3ms/step - loss: 0.1433 - accuracy: 0.9083
Epoch 89/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1249 - accuracy: 0.9242
Epoch 90/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1337 - accuracy: 0.9187
29

141/141 [==============================] - 0s 2ms/step - loss: 0.1420 - accuracy: 0.9172
Epoch 72/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1496 - accuracy: 0.9174
Epoch 73/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1484 - accuracy: 0.9137
Epoch 74/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1571 - accuracy: 0.9052
Epoch 75/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1405 - accuracy: 0.9201
Epoch 76/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1512 - accuracy: 0.9214
Epoch 77/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1409 - accuracy: 0.9209
Epoch 78/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1391 - accuracy: 0.9217
Epoch 79/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1291 - accuracy: 0.9331
Epoch 80/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1337 - accuracy: 0.9278
Ep

141/141 [==============================] - 0s 2ms/step - loss: 0.1453 - accuracy: 0.9246
Epoch 62/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1668 - accuracy: 0.9087
Epoch 63/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1528 - accuracy: 0.9147
Epoch 64/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1387 - accuracy: 0.9196
Epoch 65/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1401 - accuracy: 0.9204
Epoch 66/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1535 - accuracy: 0.9153
Epoch 67/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1512 - accuracy: 0.9114
Epoch 68/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1353 - accuracy: 0.9271
Epoch 69/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1365 - accuracy: 0.9237
Epoch 70/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1423 - accuracy: 0.9164
Ep

141/141 [==============================] - 0s 2ms/step - loss: 0.1729 - accuracy: 0.9003
Epoch 52/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1801 - accuracy: 0.8999
Epoch 53/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1801 - accuracy: 0.8952
Epoch 54/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1743 - accuracy: 0.8934
Epoch 55/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1621 - accuracy: 0.9061
Epoch 56/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1635 - accuracy: 0.9027
Epoch 57/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1577 - accuracy: 0.9157
Epoch 58/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1669 - accuracy: 0.9105
Epoch 59/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1508 - accuracy: 0.9202
Epoch 60/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1708 - accuracy: 0.9054
Ep

...

141/141 [==============================] - 0s 2ms/step - loss: 0.1496 - accuracy: 0.9012
Epoch 69/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1516 - accuracy: 0.8982
Epoch 70/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1548 - accuracy: 0.8995
Epoch 71/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1319 - accuracy: 0.9237
Epoch 72/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1539 - accuracy: 0.9027
Epoch 73/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1402 - accuracy: 0.9177
Epoch 74/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1435 - accuracy: 0.9070
Epoch 75/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1424 - accuracy: 0.9092
Epoch 76/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1278 - accuracy: 0.9271
Epoch 77/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1384 - accuracy: 0.9151
Ep

141/141 [==============================] - 0s 2ms/step - loss: 0.1647 - accuracy: 0.9074
Epoch 59/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1609 - accuracy: 0.9123
Epoch 60/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1630 - accuracy: 0.8982
Epoch 61/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2037 - accuracy: 0.8934
Epoch 62/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1615 - accuracy: 0.9098
Epoch 63/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1946 - accuracy: 0.8786
Epoch 64/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1920 - accuracy: 0.8910
Epoch 65/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1604 - accuracy: 0.9068
Epoch 66/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1660 - accuracy: 0.9033
Epoch 67/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1681 - accuracy: 0.9064
Ep

141/141 [==============================] - 0s 2ms/step - loss: 0.2257 - accuracy: 0.8805
Epoch 49/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2145 - accuracy: 0.8689
Epoch 50/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2096 - accuracy: 0.8796
Epoch 51/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1842 - accuracy: 0.8926
Epoch 52/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2179 - accuracy: 0.8717
Epoch 53/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1669 - accuracy: 0.8983
Epoch 54/90
141/141 [==============================] - 0s 3ms/step - loss: 0.1849 - accuracy: 0.8880
Epoch 55/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1815 - accuracy: 0.8849
Epoch 56/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2050 - accuracy: 0.8820
Epoch 57/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1789 - accuracy: 0.8925
Ep

141/141 [==============================] - 0s 2ms/step - loss: 0.2600 - accuracy: 0.8814
Epoch 39/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2688 - accuracy: 0.8541
Epoch 40/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1968 - accuracy: 0.8879
Epoch 41/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1999 - accuracy: 0.8870
Epoch 42/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1827 - accuracy: 0.8989
Epoch 43/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1947 - accuracy: 0.8807
Epoch 44/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1881 - accuracy: 0.8883
Epoch 45/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2255 - accuracy: 0.8759
Epoch 46/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1936 - accuracy: 0.8874
Epoch 47/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2037 - accuracy: 0.8827
Ep

141/141 [==============================] - 0s 2ms/step - loss: 0.2761 - accuracy: 0.8736
Epoch 29/90
141/141 [==============================] - 0s 2ms/step - loss: 0.3568 - accuracy: 0.8468
Epoch 30/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2470 - accuracy: 0.8826
Epoch 31/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2106 - accuracy: 0.8804
Epoch 32/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2226 - accuracy: 0.8797
Epoch 33/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2046 - accuracy: 0.8870
Epoch 34/90
141/141 [==============================] - 0s 2ms/step - loss: 0.1828 - accuracy: 0.9024
Epoch 35/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2356 - accuracy: 0.8835
Epoch 36/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2088 - accuracy: 0.8814
Epoch 37/90
141/141 [==============================] - 0s 2ms/step - loss: 0.2328 - accuracy: 0.8805
Ep

In [238]:
sum(sta3)/30

0.9282592594623565

In [239]:
sum(ste3)/30

0.9053260902563731

In [240]:
sum(sf3)/30

0.8816725456867357

In [241]:
max(sta3)

0.9415555596351624

In [242]:
max(ste3)

0.925000011920929

In [243]:
max(sf3)

0.9093298291721419

In [244]:
stdev(sta3)

0.015182730589150163

In [245]:
stdev(ste3)

0.0177560791669496

In [246]:
stdev(sf3)

0.027192870052736375

In [247]:
save=1

In [ ]:
final=1